In [1]:
import pymc3 as pm
import numpy as np


In [11]:
with pm.Model() as model:
    true_price = pm.Normal("true_price", mu=35e3, sd=75e2)
    price1=pm.Normal("price1",3000,12000)
    price2=pm.Normal("price2",500,3000)
    ans=price1+price2
    step=pm.Metropolis()
    trace=pm.sample(50000,step=step)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [price2]
>Metropolis: [price1]
>Metropolis: [true_price]
Sampling 4 chains, 0 divergences: 100%|███████████████████████████████████| 202000/202000 [02:22<00:00, 1412.85draws/s]
The number of effective samples is smaller than 25% for some parameters.


In [12]:
with model:
    burned_trace = trace[10000:]
price_trace = burned_trace["true_price"]

def showdown_loss(guess, true_price, risk):
        loss = np.zeros_like(true_price)
        ix = true_price < guess
        loss[~ix] = np.abs(guess - true_price[~ix])
        close_mask = [abs(true_price - guess) <= 250]
        loss[close_mask] = -2*true_price[close_mask]
        loss[ix] = risk
        return loss

In [13]:
import scipy.optimize as sop
guesses = np.linspace(5000, 50000, 70) 
risks = np.linspace(30000, 150000, 6)
expected_loss = lambda guess, risk: \
    showdown_loss(guess, price_trace, risk).mean()

In [14]:
for _p in risks:
    results = [expected_loss(_g, _p) for _g in guesses]

<ipython-input-12-9a2a3fff2f4b>:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  loss[close_mask] = -2*true_price[close_mask]


In [15]:
for _p in risks:
    _min_results = sop.fmin(expected_loss, 15000, args=(_p,),disp = False)
    _results = [expected_loss(_g, _p) for _g in guesses]
    print("minimum at risk %d: %.2f" % (_p, _min_results))

<ipython-input-12-9a2a3fff2f4b>:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  loss[close_mask] = -2*true_price[close_mask]


minimum at risk 30000: 27662.49
minimum at risk 54000: 23650.27
minimum at risk 78000: 22011.05
minimum at risk 102000: 21419.45
minimum at risk 126000: 20388.10
minimum at risk 150000: 19968.06
